<a href="https://colab.research.google.com/github/Onamihoang/DL01/blob/master/Culua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

In [0]:
!pip install transformers

!pip install pyfunctional

In [0]:
from transformers import BertForMaskedLM, BertTokenizer, BertModel
import torch

model_name = "bert-large-uncased"
mask_token = '***mask***'

import logging
logging.basicConfig(level = logging.INFO)

bert=BertForMaskedLM.from_pretrained(model_name)
tokenizer=BertTokenizer.from_pretrained(model_name)
bert.eval()

#bertMask = bertMask.to('cuda')

In [0]:
from functional import seq
from collections import defaultdict
from typing import Dict, List, Tuple, Union, overload

def softmax(x):
    return x.exp() / (x.exp().sum(-1)).unsqueeze(-1)



In [0]:
import numpy

def get_probs_for_words(sent,w1,w2,w3,w4):
    pre , target  ,post = sent.split('***')
    if 'mask' in target.lower():
        target = ['[MASK]']
    else:
        target = tokenizer.tokenize(target)
        
    tokens = ['[CLS]'] + tokenizer.tokenize(pre)
    target_idx = len(tokens)
    print(target_idx)
    tokens+=target+tokenizer.tokenize(post)+['[SEP]']
    print(tokens)

    input_ids=tokenizer.convert_tokens_to_ids(tokens)
    try:
        #word_token = tokenizer.tokenize([w1,w2,w3,w4])
        #print(word_token)
        word_ids = tokenizer.convert_tokens_to_ids([w1,w2,w3,w4])
        print(word_ids)
    except KeyError:
        print("skipping",w1,w2,"bad wins")
        return None
    tens = torch.LongTensor(input_ids).unsqueeze(0)
    print(tens)
    res=bert(tens)#[0,target_idx]
    preds = res[0]
    preds = softmax(preds)
    print(type(preds))
    print(preds.shape)
    #print(preds[0,)
    pred = preds[0, target_idx,word_ids]
    print(type(pred))
    print(pred.shape)
    #pred = list(preds)
    #probs = softmax(preds)
    #print(probs)

    
    print([float(x) for x in pred])


    #predicted_inds = torch.argsort(-predictions[0, masked_index])
    

masked_string = " Graduates from the Melrose College of Technology often make ***mask*** contributions in several fields, including engineering, computer sciences and astronomy. "
options = ["value", "valuable", "valuably", "valuing"]

masked_string = "Our apartment building's occupancy increased ***mask*** the efforts of our knowledgeable and friendly rental agent, Ms. Gova."
#'despite', 'due to', 'provided that', 'because'] due to despite


a = get_probs_for_words(masked_string, 'despite', 'due to','provided that','because')

11
['[CLS]', 'our', 'apartment', 'building', "'", 's', 'o', '##cc', '##up', '##ancy', 'increased', '[MASK]', 'the', 'efforts', 'of', 'our', 'knowledge', '##able', 'and', 'friendly', 'rental', 'agent', ',', 'ms', '.', 'gov', '##a', '.', '[SEP]']
[2750, 100, 100, 2138]
tensor([[  101,  2256,  4545,  2311,  1005,  1055,  1051,  9468,  6279, 11656,
          3445,   103,  1996,  4073,  1997,  2256,  3716,  3085,  1998,  5379,
         12635,  4005,  1010,  5796,  1012, 18079,  2050,  1012,   102]])
<class 'torch.Tensor'>
torch.Size([1, 29, 30522])
<class 'torch.Tensor'>
torch.Size([4])
[0.03151131793856621, 6.148987097276404e-08, 6.148987097276404e-08, 2.9065360649838112e-05]


In [0]:
masked_string = "A firm will not ***mask*** if its employees are unhappy."
options = ['prosper', 'prosperous', 'prosperity', 'prospering'] #prosper prosperous

a = get_probs_for_words(masked_string, "prosper", "prosperous","prosperity","prospering")

5
['[CLS]', 'a', 'firm', 'will', 'not', '[MASK]', 'if', 'its', 'employees', 'are', 'unhappy', '.', '[SEP]']
[100, 18241, 14165, 100]
tensor([[  101,  1037,  3813,  2097,  2025,   103,  2065,  2049,  5126,  2024,
         12511,  1012,   102]])
<class 'torch.Tensor'>
torch.Size([1, 13, 30522])
<class 'torch.Tensor'>
torch.Size([4])
[7.73514386764873e-07, 2.8502333861979423e-06, 1.3334482673599268e-06, 7.73514386764873e-07]


In [0]:

#['final', 'finalize', 'finally', 'finality'] finalize final

masked_string = "Ms. Najar wants to ***mask*** the costs by tonight."
a = get_probs_for_words(masked_string, "final", "finalize","finally","finality")

7
['[CLS]', 'ms', '.', 'na', '##jar', 'wants', 'to', '[MASK]', 'the', 'costs', 'by', 'tonight', '.', '[SEP]']
[2345, 100, 2633, 100]
tensor([[  101,  5796,  1012,  6583, 16084,  4122,  2000,   103,  1996,  5366,
          2011,  3892,  1012,   102]])
<class 'torch.Tensor'>
torch.Size([1, 14, 30522])
<class 'torch.Tensor'>
torch.Size([4])
[2.929694346676115e-05, 3.1597605811839458e-06, 1.1550563385753776e-06, 3.1597605811839458e-06]


In [0]:
masked_string = "The Calgary Inn attaches importance to the ***mask*** of its rooms and always offers good service at reasonable prices."
#['clean', 'cleanliness', 'cleanly', 'cleanest'] cleanliness clean
"cleaned": "clean",
"cleaner": "clean",
"cleaners": "cleaner",
"cleanest": "clean",
"cleaning": "clean",
"cleanings": "cleaning",
"cleanlier": "cleanly",
a = get_probs_for_words(masked_string, "cleanliness", "cleanly","cleanest","clean")

9
['[CLS]', 'the', 'calgary', 'inn', 'attache', '##s', 'importance', 'to', 'the', '[MASK]', 'of', 'its', 'rooms', 'and', 'always', 'offers', 'good', 'service', 'at', 'reasonable', 'prices', '.', '[SEP]']
[100, 100, 100, 4550]
tensor([[  101,  1996, 10112,  7601, 29489,  2015,  5197,  2000,  1996,   103,
          1997,  2049,  4734,  1998,  2467,  4107,  2204,  2326,  2012,  9608,
          7597,  1012,   102]])
<class 'torch.Tensor'>
torch.Size([1, 23, 30522])
<class 'torch.Tensor'>
torch.Size([4])
[8.846667043371781e-08, 8.846667043371781e-08, 8.846667043371781e-08, 3.41269769705832e-05]
